In [1]:
## Import Lib

import pandas as pd
from langdetect import detect
import string
import emoji
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk import word_tokenize 
from nltk.stem import WordNetLemmatizer 
nltk.download('punkt')
nltk.download('wordnet')

pd.set_option('display.max_rows',500)

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JYOTHIPRAKASH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\JYOTHIPRAKASH\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
## Load the dataset

df = pd.read_csv("2020-04-06.tsv",sep="\t",header=0)
df.head(3)

,github_repo_url,repo_description,topics,owner_repo_name,owner_name,owner_type,organization_bio,repo_created_day,primary_language_name,license_name,...,count_of_stars,count_of_watchers,count_distinct_contributors,count_contributions,count_commits,count_commit_comments,count_created_issues,count_pull_requests_created,count_pull_requests_reviews,count_comments_on_issues_and_pull_requests
0,https://github.com/CSSEGISandData/COVID-19,"novel coronavirus (covid-19) cases, provided b...","systems-science, covid-19, johns-hopkins-unive...",CSSEGISandData/COVID-19,CSSEGISandData,User,NaN,2020-02-04,NaN,NaN,...,19434,19417,2746,11609,3256,152,1669,361,119,6052
1,https://github.com/phildini/stayinghomeclub,a list of all the companies wfh or events chan...,"remote-work, covid19, covid-19, static-site",phildini/stayinghomeclub,phildini,User,NaN,2020-03-04,Ruby,cc0-1.0,...,456,453,1091,4156,1293,3,78,1350,934,498
2,https://github.com/tokyo-metropolitan-gov/covid19,東京都 新型コロナウイルス感染症対策サイト / tokyo covid-19 task fo...,covid-19,tokyo-metropolitan-gov/covid19,tokyo-metropolitan-gov,Organization,tokyo metropolitan government,2020-02-29,Vue,mit,...,5174,5165,537,11855,2774,2,906,1473,1533,5167


In [7]:
## EAA

df.describe()

,count_of_public_forks,count_of_stars,count_of_watchers,count_distinct_contributors,count_contributions,count_commits,count_commit_comments,count_created_issues,count_pull_requests_created,count_pull_requests_reviews,count_comments_on_issues_and_pull_requests
count,36295.000000,36295.000000,36295.000000,36295.000000,36295.000000,36295.000000,36295.000000,36295.000000,36295.000000,36295.000000,36295.000000
mean,1.161813,3.205345,3.191073,1.656427,28.674446,23.978317,0.023832,0.882050,1.188511,0.691583,1.910153
std,56.341646,119.499867,119.379832,16.832495,803.767059,795.372879,0.863634,17.416855,14.217458,14.950501,49.483063
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,1.000000,5.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,1.000000,16.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,10184.000000,19434.000000,19417.000000,2746.000000,150065.000000,150065.000000,152.000000,2329.000000,1473.000000,1533.000000,6052.000000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36295 entries, 0 to 36294
Data columns (total 28 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   github_repo_url                             36295 non-null  object
 1   repo_description                            23132 non-null  object
 2   topics                                      3276 non-null   object
 3   owner_repo_name                             36295 non-null  object
 4   owner_name                                  36295 non-null  object
 5   owner_type                                  36295 non-null  object
 6   organization_bio                            1666 non-null   object
 7   repo_created_day                            36295 non-null  object
 8   primary_language_name                       28071 non-null  object
 9   license_name                                8030 non-null   object
 10  is_github_pages       

In [9]:
df.isna().sum()

github_repo_url                                   0
repo_description                              13163
topics                                        33019
owner_repo_name                                   0
owner_name                                        0
owner_type                                        0
organization_bio                              34629
repo_created_day                                  0
primary_language_name                          8224
license_name                                  28265
is_github_pages                                   0
has_readme                                        0
has_wiki                                          0
has_merged_prs                                    0
has_issues                                        0
has_contributor_guide                             0
has_code_of_conduct                               0
count_of_public_forks                             0
count_of_stars                                    0
count_of_wat

In [10]:
## Clean the dataset and Filter the data based on our requirments

df_filtered = df[
    (df["has_merged_prs"]==True) &
    (pd.isna(df["repo_description"])==False) &
    (pd.isna(df["primary_language_name"])==False) &
    (df["count_distinct_contributors"]>=2) &
    (df["count_commits"]>=5)    
     ]

In [11]:
# df.shape->(36295, 28)  after filtering from the dataset 
df_filtered.shape

(1856, 28)

In [12]:
## Data Clening

# Detect languages with error handling
def dectect_and_hadle_error(x):
    try:
        return detect(x)
    except:
        return "Error"
    
# Check for latin characters
def has_only_latin_letters(text):
    ch_set = string.printable+"-"
    return all(True if x in ch_set else False for x in text)

# Remove punctuations
def remove_punctuations(text):
    punctuation_list = string.punctuation + '-' 
    return text.translate(str.maketrans('', '', punctuation_list))

## Full set of processing

In [13]:
## Full set of processing 

# 1.language
df_filtered["laguage"] = df_filtered["repo_description"].apply(lambda x: "None" if pd.isna(x) else dectect_and_hadle_error(str(x)))
# print("Total no. Lang:",df_filtered["laguage"].nunique())

df_filtered = df_filtered[df_filtered["laguage"]=="en"].copy()

In [14]:
df_filtered["is_latin"] = df_filtered["repo_description"].apply(lambda x: has_only_latin_letters(emoji.demojize(x)))
df_filtered = df_filtered[df_filtered["is_latin"]==True].copy()

In [15]:
# 2.Clean the description, topic, and languages into a big bag of words

df_filtered["repo_description_cleaned"] = df_filtered["repo_description"].apply(lambda x: remove_punctuations(x))
df_filtered["topics"] = df_filtered.apply(lambda x: remove_punctuations(str(x['topics']).replace(',','').replace('nan','')), axis=1)
df_filtered["topics"].fillna("",inplace=True)

In [16]:
df_filtered["repo_description_cleaned"]

1       a list of all the companies wfh or events chan...
3                       📊 source code of the main website
5       api for current cases and more stuff about cov...
8       🦠 a simple and fast  200ms api for tracking th...
9       🦠useful projects and resources for covid19 201...
                              ...                        
5688    coronavirus is worse than the flu unless you d...
5689    app to help our church meet the needs of the c...
5699       covid19againsthumanity hackathon frontend repo
5719    interactive visualization of weekly and daily ...
5724                                    searo him covid19
Name: repo_description_cleaned, Length: 1428, dtype: object

In [17]:
df_filtered["topics"]

1                   remotework covid19 covid19 staticsite
3       covid19india tracker visualization covid19 ana...
5                                                        
8       heroku recoveries coronavirus pipenv covid19 p...
9       corona awesomecoronavirus virus coronavirusinf...
                              ...                        
5688                                                     
5689                                                     
5699                                                     
5719    coronavirus covidcovid19datavisualization covi...
5724                                                     
Name: topics, Length: 1428, dtype: object

In [18]:
# Now that we have cleaned and filterded the dataset 
# We can transform that huge-chunk of data into one sigle bucket for each repo

df_filtered["repo_description_plus_topic"] = (df_filtered["repo_description_cleaned"]
                                              +" "+df_filtered["topics"]+" "+df_filtered["primary_language_name"])

In [19]:
df_filtered.reset_index(drop=True, inplace=True)

In [20]:
repo_lookup = df_filtered.copy()

# Natural Language Processing and model building

In [21]:
## Tokanization

class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self,articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [22]:
# remove the stop words i.e., a,an,is.. and covid-19 related words

stopwords = list(ENGLISH_STOP_WORDS)+['covid19','coronavirus','virus',
                                      'corona','covid','pandemic','sarscov2','outbreak','19','disease',
                                      '2019','2019ncov','cord19','repository','repo','2020','20','covid2019','covidvirus',
                                      'cases','case']
# Creat a count vectorizer of ngram

word_vectorizer = CountVectorizer(ngram_range=(1,1), analyzer='word',stop_words=stopwords, tokenizer=LemmaTokenizer())


# Fit the vectorizer on the existing list of repos and creat a confusion matrix

sparse_vector_matrix = word_vectorizer.fit_transform(df_filtered['repo_description_plus_topic'])

In [23]:
print(sparse_vector_matrix)

  (0, 1819)	1
  (0, 543)	1
  (0, 3313)	1
  (0, 1144)	1
  (0, 423)	1
  (0, 2557)	1
  (0, 2901)	1
  (0, 2646)	1
  (1, 3418)	1
  (1, 2846)	1
  (1, 501)	1
  (1, 1870)	1
  (1, 3303)	1
  (1, 725)	1
  (1, 3102)	1
  (1, 3250)	1
  (1, 123)	1
  (1, 1695)	1
  (2, 1695)	1
  (2, 152)	1
  (2, 844)	1
  (2, 2948)	1
  (2, 2127)	1
  (2, 2927)	1
  (3, 2846)	1
  :	:
  (1423, 3155)	1
  (1423, 1251)	1
  (1423, 3358)	1
  (1423, 3177)	1
  (1424, 1695)	1
  (1424, 162)	1
  (1424, 1459)	1
  (1424, 2068)	1
  (1424, 539)	1
  (1424, 1934)	1
  (1424, 455)	1
  (1425, 1695)	1
  (1425, 1423)	1
  (1425, 1290)	1
  (1425, 699)	1
  (1426, 3250)	1
  (1426, 1695)	1
  (1426, 684)	1
  (1426, 860)	1
  (1426, 1642)	1
  (1426, 422)	1
  (1426, 3307)	1
  (1426, 769)	1
  (1427, 1695)	1
  (1427, 2717)	1


In [24]:
## Build prediction function

def text_recommender(input_df, word_vectorizer=word_vectorizer,  sparse_vector_matrix = sparse_vector_matrix, repo_lookup=repo_lookup):
    
    input_df["bag_of_words"] = input_df.apply(lambda x: " ".join(x), axis=1)
    
    # vectorize the inputted string
    inputted_vector = word_vectorizer.transform(input_df['bag_of_words'])
    
    # calculate the cosine similarity
    one_dimension_cosine_sim = cosine_similarity(inputted_vector, sparse_vector_matrix)
    
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(one_dimension_cosine_sim[0]).sort_values(ascending = False)
    
    # only show matches that have some similarity
    score_series = score_series[score_series>0]
    
    # getting the indexes of the 10 most similar repos
    top_10_indexes = list(score_series.iloc[1:10].index)
    
    # return the list of recommended repo
    recommended_repos = repo_lookup.loc[top_10_indexes]
    
    return recommended_repos

In [25]:
## Prediction

class RepoRecomender(object):
  
    def __init__(self,word_vectorizer,sparse_vector_matrix,repo_lookup,text_recommender):
        self.word_vectorizer = word_vectorizer 
        self.sparse_vector_matrix = sparse_vector_matrix
        self.repo_lookup = repo_lookup
        self.text_recommender = text_recommender
        
    def predict(self,context,model_input):
        output_df = self.text_recommender(model_input)
        return [output_df.to_dict('records')]

In [44]:
## Model Testing::

model = RepoRecomender(word_vectorizer = word_vectorizer,
                                       sparse_vector_matrix = sparse_vector_matrix,
                                       repo_lookup = repo_lookup,
                                       text_recommender = text_recommender)

model_input = pd.DataFrame([["Java","Kotlin", "Android"]])
model_output = model.predict(None,model_input)

print(model_output)

[[{'github_repo_url': 'https://github.com/COVID-19-PROJECT/COVID-19-Android', 'repo_description': 'covid android project', 'topics': '', 'owner_repo_name': 'COVID-19-PROJECT/COVID-19-Android', 'owner_name': 'COVID-19-PROJECT', 'owner_type': 'Organization', 'organization_bio': nan, 'repo_created_day': '2020-03-23', 'primary_language_name': 'Kotlin', 'license_name': nan, 'is_github_pages': False, 'has_readme': True, 'has_wiki': True, 'has_merged_prs': True, 'has_issues': True, 'has_contributor_guide': False, 'has_code_of_conduct': False, 'count_of_public_forks': 0, 'count_of_stars': 0, 'count_of_watchers': 0, 'count_distinct_contributors': 4, 'count_contributions': 81, 'count_commits': 60, 'count_commit_comments': 0, 'count_created_issues': 0, 'count_pull_requests_created': 8, 'count_pull_requests_reviews': 13, 'count_comments_on_issues_and_pull_requests': 0, 'laguage': 'en', 'is_latin': True, 'repo_description_cleaned': 'covid android project', 'repo_description_plus_topic': 'covid andr

# Save the Model

In [45]:
import pickle

In [46]:
filename = "model/nltk_model.pkl"
pickle.dump(model,open(filename,"wb"))

In [48]:
model = pickle.load(open("model/nltk_model.pkl","rb"))
# model.predict(None,pd.DataFrame([["Java","Kotlin", "Android"]]))

pickle.dump(word_vectorizer,open("model/word_vectorizer.pkl","wb"))
# pickle.load(open("model/word_vectorizer.pkl","rb"))

pickle.dump(sparse_vector_matrix,open("model/sparse_vector_matrix.pkl","wb"))
# print(pickle.load(open("model/sparse_vector_matrix.pkl","rb")))

pickle.dump(repo_lookup,open("model/repo_lookup.pkl","wb"))
# pickle.load(open("model/repo_lookup.pkl","rb"))

In [68]:
pickle.dump(stopwords,open("model/stopwords.pkl","wb"))

['hundred',
 'anyway',
 'every',
 'since',
 'whatever',
 'below',
 'became',
 'someone',
 'thence',
 'am',
 'thin',
 'former',
 'fifteen',
 'more',
 'of',
 'during',
 'seemed',
 'nowhere',
 'within',
 'being',
 'others',
 'bill',
 'because',
 'which',
 'all',
 'otherwise',
 'these',
 'none',
 'without',
 'thereby',
 'latter',
 'serious',
 'her',
 'this',
 'now',
 'already',
 'twenty',
 'do',
 'well',
 'un',
 'too',
 'much',
 'upon',
 'eg',
 'down',
 'you',
 'across',
 'eight',
 'either',
 'cant',
 'had',
 'last',
 'the',
 'still',
 'everywhere',
 'hence',
 'except',
 'into',
 'between',
 'so',
 'a',
 'then',
 'thus',
 'where',
 'hasnt',
 'nor',
 'himself',
 'system',
 'by',
 'sometimes',
 'elsewhere',
 'themselves',
 'up',
 'five',
 'over',
 'eleven',
 'i',
 'whoever',
 'together',
 'thick',
 'anyone',
 'whose',
 'very',
 'ours',
 'those',
 'it',
 'herself',
 'there',
 'fill',
 'rather',
 'thereupon',
 'should',
 'beside',
 'anywhere',
 'nevertheless',
 'who',
 'mostly',
 'both',
 'suc